# How to Find Investor Buyer Leads using REAPI
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1u0CMHKEpQAtwwLQq0pw5tH2iRzKlfxlN?usp=sharing)

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |
| External References   | API |
| Input Datasets        | Source name |
| Output Datasets       | Source name |
| Input Data Source     | Pandas DataFrame |
| Output Data Source    | Pandas DataFrame |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 15th Jun 2024 | Ariel Herrera | Create notebook. |

## Getting Started
1. Copy this notebook -> File -> Save a Copy in Drive
2. Directions

## Useful Resources
- [Google Colab Cheat Sheet](https://towardsdatascience.com/cheat-sheet-for-google-colab-63853778c093)
- [Redfin Housing Market Overview](https://www.redfin.com/city/13655/FL/Orlando/housing-market)

## Why Agents Should Focus on Investor Buyer Leads 🤔
1. Investor buyers can be <b>repeat buyers</b> 🔁 for consistent income
2. Investor buyers can unlock <b>new income streams</b> 💰
  - market research fees
  - property management commissions
  - creative deal equity ownership
3. Investor buyers are relatively <b>easy leads to locate</b> 🔍 via real estate data sources
4. Investor buyers are <b>lower competition</b> 📉 leads than traditional buyers (i.e. first time home buyer)

## <font color="blue">Install Packages</font>

## <font color="blue">Imports</font>

In [5]:
import requests
from getpass import getpass
import pandas as pd

## <font color="blue">Functions</font>

## <font color="blue">Locals & Constants</font>

In [6]:
# census api key - https://api.census.gov/data/key_signup.html
reapi_api_key = getpass('Enter the api key: ')

Enter the api key: ··········


## <font color="blue">Data</font>

In [7]:
url = "https://api.realestateapi.com/v2/PropertySearch"

payload = {
    #|-------INITIAL PARAMS-------|#
    "ids_only": False, # False to return all property details
    "obfuscate": False, # False to show names & address
    "summary": False, # True for aggregate values, False for individual properties
    "count": False, # True to return count of results
    "size": 250, # number of properties to return (max = 250 properties)

    #|----------FILTERS-----------|#
    # location
    "city": "Orlando",
    "state": "FL",

    # property details
    "property_type": "SFR", # property type - single family residence
    "year_built_max": 2020, # max date year built - remove new construction (likely second home not rental)
    "value_max": 500000, # max estimated value of property

    # owner details
    "properties_owned_max": 10, # total number of properties owned (likely newer investors without established team)
    "years_owned_max": 1, # number of years owned property - recently purchased
    "corporate_owned": False, # owned by individual NOT a corproration (i.e. LLC)
    "or": [
        {"out_of_state_owner": True}, # out of state owner (not primary home)
        {"absentee_owner": True} # out of state owner (not primary home)
    ]
}
headers = {
    "accept": "application/json",
    "x-user-id": "UniqueUserIdentifier",
    "content-type": "application/json",
    "x-api-key": reapi_api_key
}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

{"live":true,"input":{"ids_only":false,"obfuscate":false,"summary":false,"count":false,"size":50,"city":"Orlando","state":"FL","property_type":"SFR","year_built_max":2020,"value_max":500000,"properties_owned_max":10,"years_owned_max":1,"corporate_owned":false,"or":[{"out_of_state_owner":true},{"absentee_owner":true}],"sort":{}},"data":[{"absenteeOwner":true,"address":{"address":"6125 Apollos Corner Way, Orlando, FL 32829","city":"Orlando","county":"Orange","state":"FL","street":"6125 Apollos Corner Way","zip":"32829"},"adjustableRate":false,"airConditioningAvailable":false,"apn":"19-23-31-2006-00-650","assessedImprovementValue":190166,"assessedLandValue":65000,"assessedValue":255166,"assumable":false,"auction":false,"auctionDate":null,"basement":false,"bathrooms":2,"bedrooms":3,"cashBuyer":false,"corporateOwned":false,"death":false,"deck":false,"deckArea":0,"documentType":"Intrafamily Transfer","documentTypeCode":"DTIT","equity":false,"equityPercent":78,"estimatedEquity":231086,"estima

In [8]:
print('Number of properties:', response.json()['resultCount'])

Number of properties: 84


## <font color="blue">Transformations</font>

Transform JSON obj to DataFrame (tabular dataset with rows and columns)

In [9]:
trunc_l = [] # create empty list

# iterate through all properties
for x in response.json()['data']:
    # create dictionary for each property
    d = {
        # owner
        'owner1FirstName': x['owner1FirstName'] if 'owner1FirstName' in x else None,
        'owner1LastName': x['owner1LastName'] if 'owner1LastName' in x else None,
        # number of properties
        'totalPropertiesOwned': x['totalPropertiesOwned'] if 'totalPropertiesOwned' in x else None,
        # last purchased
        'lastSaleDate': x['lastSaleDate'] if 'lastSaleDate' in x else None,
        # equity
        'equityPercent': x['equityPercent'] if 'equityPercent' in x else None,
        'yearBuilt': x['yearBuilt'] if 'yearBuilt' in x else None,
        'yearsOwned': x['yearsOwned'] if 'yearsOwned' in x else None,
        # contact
        'address': x['address']['address'] if 'address' in x else None,
        'mailAddress': x['mailAddress']['address'] if 'mailAddress' in x else None
    }
    # append dictionary to list
    trunc_l.append(d)

In [10]:
# transform list of dictionaries to a dataframe
df = pd.DataFrame(trunc_l)
# sort values by last sale date
df.sort_values(by='lastSaleDate', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)
df

,owner1FirstName,owner1LastName,totalPropertiesOwned,lastSaleDate,equityPercent,yearBuilt,yearsOwned,address,mailAddress
0,Dejan,Samardzic,4,2023-09-12,38,1987,0,"14434 Mandolin Dr, Orlando, FL 32837","1965 Bridgewater Dr, Lake Mary, Fl 32746"
1,Sira,Nakary,2,2023-09-11,81,1969,0,"4417 Prince Hall Blvd, Orlando, FL 32811","4652 Campsite Loop, Orlando, Fl 32824"
2,Larry,Pupo,3,2023-09-08,19,1958,0,"5656 Pecos St, Orlando, FL 32807","1938 Colton Dr, Orlando, Fl 32822"
3,Hans,Michielssen,2,2023-08-31,49,2007,0,"9120 Silverback Ln, Orlando, FL 32827","6378 Westcott Cove Blvd, Orlando, Fl 32829"
4,Bush,Travis,4,2023-08-31,100,1977,0,"10014 Brownwood Ave, Orlando, FL 32825","208 Orchid Dr, Greenville, Sc 29617"
5,Delgado,Quinones,2,2023-08-29,11,1963,0,"4125 Lake Lawne Ave, Orlando, FL 32808","4215 Mendenwood Ln, Orlando, Fl 32826"
6,Patronella,Prescod,4,2023-08-26,94,1999,0,"808 Magnolia Creek Cir, Orlando, FL 32828","525 Beach 65th St, Arverne, Ny 11692"
7,Ignacio,Vazquez Aviles,3,2023-08-24,25,2013,0,"10305 Belfry Cir, Orlando, FL 32832","13581 Larkton Ln, Orlando, Fl 32832"
8,Grant,Passwater,8,2023-08-18,37,2019,0,"14344 Swanley St, Orlando, FL 32832","13224 Puff Rd, Fort Wayne, In 46845"
9,Sophia,Roxton,2,2023-08-18,100,1952,0,"3714 Starks St, Orlando, FL 32805","6438 Royal Tern St, Orlando, Fl 32810"


## <font color="blue"> Sample Mailer or E-mail</font>

Subject: Unlock Hidden Gems in the [City] Real Estate Market

Hi [First Name],

Congratulations on consistently growing your investment portfolio! As an investor-friendly agent here in [City], I wanted to connect because I understand the importance of finding the right deals in today's market.

I work closely with real estate investors like yourself, providing them with:

- <b>Market Expertise</b>: I stay ahead of the curve on [City]'s real estate trends, giving you access to valuable insights that can help you identify hidden opportunities and make informed investment decisions.
- <b>Deal Analysis</b>: I go beyond just showing properties. I'll help you analyze potential investments, considering factors like rental income, potential appreciation, and overall return on investment.
- <b>Streamlined Transactions</b>: From negotiation to closing, I'll handle the nitty-gritty details to ensure a smooth and successful transaction.
Free Monthly Market Report:

To empower your investment decisions, <b>I offer a free monthly newsletter with data-driven insights on [City]'s real estate market performance</b>, including rental trends, cap rates, and neighborhood analysis. You can see a sample attached to get a feel for the valuable information you'll receive.

There's no pressure to sign up, but if you're interested, you can scan the QR code below to subscribe for free – I promise no spam!

I'm confident I can be a valuable resource as you continue to expand your investment portfolio in [City]. Feel free to reach out if you have any questions or would like to discuss your investment goals in more detail.

Best regards,

[Your Name]

Investor-Friendly Real Estate Agent

[Your Contact Information]

P.S.  I also have access to off-market properties that may not be publicly listed, giving you an extra edge in finding the best deals.

![](https://drive.google.com/uc?export=view&id=1OL16Lm-7vSqzeLDBDbCNWIbJlYx2ciYd)

## <font color="blue">CTA: How to Get Your FREE Investor Buyer Leads List 👇</font>
Book a 20 minute call with me [here](https://calendly.com/analyticsariel/agent-free-investor-buyer-leads) 📅

# End Notebook